__Author__: Bogdan Bintu, Modified by Adam Jussila

__Date__: 01/28/2025

__Scope__: Analysis tool Ren Lab

<h3>Pipeline setup</h3>

This pipeline requires the following additional packages to be installed in the python environment: pytorch, py-opencv

These can be installed with this command in the anaconda prompt:

`conda install pytorch py-opencv`

The environment all data analyzed in this manuscript were run with is attached using the environment_ChromTracer_Sox2.yml

<h4>Set the folder with the raw data</h4>

In [ ]:
master_folders = [r'/path/to/raw/data']

<h4>Set the folder for the analysis results</h4>

In [ ]:
analysis_folder = r'/path/to/analysis/folder'

<h4>Specify colors used in this experiment</h4>

In [ ]:
colors = ['750','647', '561', 'beads']

<h3>Initialize the pipeline</h3>

In [ ]:
import sys
import copy

sys.path.append(r'/path/to/CommonTools')
import DomainTools as dt

import PipelineFunctions as pipeline
import matplotlib.pylab as plt
from scipy.spatial.distance import squareform,pdist,cdist
from scipy.stats import ranksums

experiment_name, em_threshold = pipeline.load_params(analysis_folder)

folders, fovs, h0_folder = pipeline.load_data(master_folders, analysis_folder)

In [ ]:
import glob, os
import IOTools as io
import numpy as np

<h3>Nuclei segmentation</h3>

In [ ]:
import os

#Pattern matching all DAX files to segment
dax_files = master_folders[0]+os.sep+r"H0B,B,B\Conv_zscan_*.dax"
save_dir = analysis_folder+os.sep+r"segmentation"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
#How many channels are in the dax files
dax_channels = 5
#Which channel to use for segmentation (counts from 0)
channel = 4
#Which z-slice to use for segmentation (counts from 0)
zslice = 15
###

import glob

files = list(sorted([f for f in glob.glob(dax_files)]))
print(f"Found {len(files)} files to segment")

In [ ]:
from daxfile import DaxFile
from tqdm import tqdm_notebook as tqdm

print("Loading images")
imgs = [DaxFile(filename, num_channels=dax_channels).zslice(zslice, channel) for filename in tqdm(files)]

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure(figsize=(15,6))
for k,img in enumerate(imgs[0:3]):
    plt.subplot(1,3,k+1)
    plt.axis('off')
    plt.imshow(img, vmax=np.percentile(img, 99), cmap='gray')
plt.tight_layout()

In [ ]:
# RUN CELLPOSE
import os
from cellpose import models, io
from skimage.segmentation import expand_labels

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=False, model_type='nuclei')

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus 
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus
#channels = [2,3]

# or if you have different types of channels in each image
#channels = [[2,3], [0,0], [0,0]]

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended) 
# diameter can be a list or a single number for all images
# heart diameter = 80
# U2OS diameter = 320

masks, flows, styles, diams = model.eval(imgs, diameter=80, channels=channels, min_size=1000)

#Filter tiny cells (artifacts) and expand by 5 pixels to smooth edges
newmasks = []
for mask in tqdm(masks):
    newmask = mask.copy()
    newmask = expand_labels(newmask, 3)
    newmasks.append(newmask)

savefiles = [os.path.join(save_dir, os.path.basename(file)) for file in files]
#os.makedirs(save_dir)
# save results so you can load in gui
io.masks_flows_to_seg(imgs, newmasks, flows, diams, savefiles, channels)
# save results as png
io.save_to_png(imgs, newmasks, flows, savefiles)

### Compute the flat field correction

In [ ]:
#Fast, 1-2 seconds per FOV
im_meds = pipeline.save_median_image_across_fovs(folders, analysis_folder)

In [ ]:
#Visualize median images
im = im_meds[3] # 0-3 is [750, 647, 561, 488] nm (laser wavelength)
plt.figure()
plt.imshow(im,vmax=8000);

In [ ]:
#This step takes a long time (~30 minutes per FOV for an experiment with 45 probes)
pipeline.flat_field_correction(folders, analysis_folder, fovs, colors)

<h3>Select candidate spots</h3>

In [ ]:
# select two best colors (0 is first color in imaging round, 1 is second, etc.)
colors_to_use = [0, 1]

In [ ]:
import glob
import os
import numpy as np
# load the masks if they are not already in the notebook
segmentation_folder = analysis_folder+os.sep+r'segmentation'
imgs = glob.glob(segmentation_folder+os.sep+r'Conv_zscan_*_seg.npy')
masks = np.array([ np.load(img, allow_pickle=True).item()['masks'].T for img in imgs ])
masks.shape

In [ ]:
# visualize the segmentation of one fov
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
fov_num = 7

fig = plt.figure(figsize=(10,8))
plt.imshow(masks[fov_num], cmap='seismic')
plt.contour(masks[fov_num], [0.5+x for x in range(np.max(masks[fov_num]))])

In [ ]:
#~70 seconds per FOV
pipeline.get_candidate_spots(analysis_folder, masks, colors=colors_to_use)

<h4>Check PNG files with names ending in "spots_selected_final" in the analysis folder to view candidate spots selected</h4>

<h3>Drift correction</h3>

In [ ]:
import pickle
import os
import numpy as np
import importlib
importlib.reload(pipeline)
dic = pickle.load(open(analysis_folder+os.sep+'Selected_Spot.pkl','rb'))
fovs = np.unique(dic['class_ids'])

In [ ]:
#The 'jobs' parameter is how many FOVs to run simultaneously. 
#Each batch of jobs takes a few hours. No progress bar or any output is shown while running.
pipeline.drift_correction_batch(folders, fovs, analysis_folder, colors, jobs=25)

In [ ]:
drift_fls = glob.glob(analysis_folder+os.sep+r'*_drift.npy')
drifts = []
for fl in drift_fls:
    drifts.append(np.load(fl))
drift = np.array(drifts)
drift

### Perform EM algorithm

In [ ]:
import importlib
importlib.reload(pipeline)

In [ ]:
#Pretty fast, ~10 minutes
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
rounds = 42
zxys_f, hs_f, cols_f, snrs_f, scores_f, all_scores_f, dic_spots, dic_drifts, dic_cell_ids, fovs_spots = pipeline.em_algorithm(analysis_folder, colors, rounds, blank_rounds=[25],
                                                                                                                    chromatic_correction=r'D:\ChromaticCorrection\chromatic_corr_BB_5_28_2020.pkl')

#### Save the data out so we can skip running EM in the future

In [ ]:
np.save(analysis_folder+os.sep+r'em_alg_zxys_f.npy', zxys_f)
np.save(analysis_folder+os.sep+r'em_alg_hs_f.npy', hs_f)
np.save(analysis_folder+os.sep+r'em_alg_cols_f.npy', cols_f)
np.save(analysis_folder+os.sep+r'em_alg_snrs_f.npy', snrs_f)
np.save(analysis_folder+os.sep+r'em_alg_scores_f.npy', scores_f)
np.save(analysis_folder+os.sep+r'em_alg_all_scores_f.npy', all_scores_f)
np.save(analysis_folder+os.sep+r'em_alg_dic_spots.npy', dic_spots)
np.save(analysis_folder+os.sep+r'em_alg_dic_drifts.npy', dic_drifts)
np.save(analysis_folder+os.sep+r'em_alg_dic_cell_ids.npy', dic_cell_ids)
np.save(analysis_folder+os.sep+r'em_alg_fovs_spots.npy', fovs_spots)

#### Load in the EM data if we have previously saved it out.

In [ ]:
import os
import numpy as np
zxys_f = np.load(analysis_folder+os.sep+r'em_alg_zxys_f.npy', allow_pickle=True)
hs_f = np.load(analysis_folder+os.sep+r'em_alg_hs_f.npy', allow_pickle=True)
cols_f = np.load(analysis_folder+os.sep+r'em_alg_cols_f.npy', allow_pickle=True)
snrs_f = np.load(analysis_folder+os.sep+r'em_alg_snrs_f.npy', allow_pickle=True)
scores_f = np.load(analysis_folder+os.sep+r'em_alg_scores_f.npy', allow_pickle=True)
all_scores_f = np.load(analysis_folder+os.sep+r'em_alg_all_scores_f.npy', allow_pickle=True)
dic_spots = np.load(analysis_folder+os.sep+r'em_alg_dic_spots.npy', allow_pickle=True)
dic_drifts = np.load(analysis_folder+os.sep+r'em_alg_dic_drifts.npy', allow_pickle=True)
dic_cell_ids = np.load(analysis_folder+os.sep+r'em_alg_dic_cell_ids.npy', allow_pickle=True)
fovs_spots = np.load(analysis_folder+os.sep+r'em_alg_fovs_spots.npy', allow_pickle=True)

In [ ]:
import numpy as np
import os
import FittingTools as ft
import pickle

#### Plot formatting

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['font.family']='Arial'

#### Make a histogram of scores for the data

In [ ]:
em_threshold

In [ ]:
import numpy as np

analysis_name = 'scratch'
post_analysis_folder = pipeline.make_post_analysis_folder(analysis_folder, analysis_name)

#########
# a plot showing the overlap of the rejected spots (orange) versus the accepted ones (blue).
# the blue and orange should be separated susbstiantially, and we choose our threshold below based on this. (log scale) 

# Adjusting this cutoff 
cutoff_exp = -em_threshold

#########

scores_all_rav = []
nlen = np.max(list(map(len,all_scores_f)))
scores_all_reg = [[] for i in range(nlen)]
scores_all_bad = [[] for i in range(nlen)]
for scores_cell in all_scores_f:
    for ireg,scores_reg in enumerate(scores_cell):
        scores_all_reg[ireg].extend(scores_reg)
        scores_all_rav.extend(np.sort(scores_reg)[-4:-1]) # the 3 remaining spots after selecting the best
        scores_all_bad[ireg].extend(np.sort(scores_reg)[:-1])
vec = np.ravel(np.array(scores_f))
vec = vec[np.isnan(vec)==False]

fr = 1.*np.sum(scores_all_rav>np.exp(cutoff_exp))/len(scores_all_rav)
txt_ = "Est. FP: "+str(np.round(fr*100,2))+'%'

fig = plt.figure()
plt.hist(np.log(vec),bins=1000,density=True,label='Accepted')
plt.hist(np.log(scores_all_rav),bins=1000,alpha=0.5,density=True,label='Rejected')
plt.vlines(cutoff_exp, 0, 0.5, 'r', linestyles='dashed')

plt.xlabel("ln P-value")
plt.ylabel("Probability density")
plt.legend()
plt.xlim((-19, 0))

plt.title("Normalized Histogram of P-Values \n for "+experiment_name+'\n Threshold:'+str(cutoff_exp)+", "+txt_);
fig.savefig(post_analysis_folder+'histogram_p-values.pdf',bbox_inches='tight')
fig.savefig(post_analysis_folder+'histogram_p-values.png',bbox_inches='tight')

<h4>Filter candidate chromosomes</h4>

In [ ]:
def estimate_fp_fn(coords, cell_ids, rnd=25):
    # estimate the number of cells with 1 detection, 0 detections, and 2 detections at this stage.
    cell_allele_state = []
    for cid in np.unique(cell_ids):
        locs = np.where(cell_ids == cid)
        allele_presence = 1*[ (~np.isnan(coords[loc,rnd,0])) for loc in locs ][0]
        cell_allele_state.append(allele_presence)
    counts = [0,0,0]
    for cell in cell_allele_state:
        counts[sum(cell)] += 1
    return counts

In [ ]:
############
number_of_segments = 42
max_segments_missing = 20
############

keep, zxys_clean_filtered, cell_ids_clean, hs_clean, scores_clean, fovs_spots_keep = pipeline.filter_chromosomes(42, max_segments_missing, zxys_f, hs_f, cols_f, scores_f, all_scores_f, dic_cell_ids, fovs_spots, cutoff_exp)
print("Chromosomes after filtering: "+str(len(zxys_clean_filtered)))
zxys_clean_filtered, cell_ids_clean, chrs_keep = pipeline.top_two_chrs(zxys_clean_filtered, cell_ids_clean)
hs_clean = hs_clean[chrs_keep]
scores_clean = scores_clean[chrs_keep]
fovs_spots_keep = fovs_spots_keep[chrs_keep]

In [ ]:
# estimate the number of cells with 1 detection, 0 detections, and 2 detections at this stage.
counts = estimate_fp_fn(zxys_clean_filtered, cell_ids_clean)
print("There are %d cells with a false negative, %d cells with one of each allele, and %d cells with a false positive." %(counts[0], counts[1], counts[2]))

<h3>Plot detection efficiency</h3>

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
total_len = float(len(zxys_clean_filtered))
eff = [ 1-np.divide(float(np.sum(np.isnan(np.array(zxys_clean_filtered)[:,i,0]),axis=0)),total_len) 
       for i in range(zxys_clean_filtered.shape[1]) ]
plt.bar(range(0,42),eff)
plt.ylim((0.0,1.0))
plt.hlines(np.mean(eff), -1, 42, colors='r', linestyle='dashed')
plt.ylabel('Detection Efficiency (Fraction)', size=14)
plt.xlabel('Imaging Round', size=14)
plt.title("Detection efficiency by imaging round\n for "+experiment_name, size=24)
plt.text(18,0.9,"Average Efficiency: "+str(100*np.round(np.mean(eff),2))+"%", size=14)
plt.show()
plt.tight_layout()

fig.savefig(post_analysis_folder+'detection_efficiency.pdf',bbox_inches='tight')
fig.savefig(post_analysis_folder+'detection_efficiency.png',bbox_inches='tight')

### Median distance

In [ ]:
#########
vmin = 0  #Bottom of color scale (in nm)
vmax = 700  #Top of color scale (in nm)
#########

import os

zxys_clean_ = (zxys_clean_filtered)
mats_clean = np.array(list(map(squareform,list(map(pdist,zxys_clean_)))))

im = np.nanmedian(mats_clean[:,:42,:42],axis=0)# take the median image for the representative heatmap

%matplotlib inline
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.title('Median distance for\n'+experiment_name+',\n'+str(len(zxys_clean_))+' chromosomes', size=24)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.imshow(im,vmin=vmin,vmax=vmax,cmap='seismic_r')
plt.colorbar()
fig.savefig(post_analysis_folder+'median_distance.pdf',bbox_inches='tight')
fig.savefig(post_analysis_folder+'median_distance.png',bbox_inches='tight')
np.save(analysis_folder+os.sep+r'median_distance_matrix.npy', im)

### Contact map

In [ ]:
########
cutoff = 250  #Distance (nm) threshold for a contact
########

%matplotlib inline
mat_ = 1.*np.sum(mats_clean<cutoff,axis=0)/np.sum(np.isnan(mats_clean),axis=0)

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.title('Contact map - cutoff:'+str(cutoff)+'nm\n for '+experiment_name+'\n'+str(len(mats_clean))+' chromosomes', size=24)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.imshow(np.log(mat_),cmap='seismic')
plt.colorbar()
fig.savefig(post_analysis_folder+'contact_map.pdf',bbox_inches='tight')
fig.savefig(post_analysis_folder+'contact_map.png',bbox_inches='tight')
np.save(analysis_folder+os.sep+r'AJ_combine_matrix_contact.npy', mats_clean)

<h4>Store some results in case they're needed later</h4>

In [ ]:
save_file = post_analysis_folder+'saved_info.pkl'
dic_save = {'master_folders':master_folders,'analysis_folder':analysis_folder,
            'hs_f':np.array(hs_f),'zxys_f':np.array(zxys_f),
            'scores_f':np.array(scores_f),
            'fov_ids':np.array(fovs),
            'hs_clean':hs_clean,
            'keep_chromosomes':keep,
            'log_score_cutoff':cutoff_exp,
           'fovs_spots_keep':fovs_spots_keep,'zxys_clean_filtered':zxys_clean_filtered,
           'scores_clean':scores_clean,
           'scores_all_rav':scores_all_rav}
pickle.dump(dic_save,open(save_file,'wb'))

### Check data

In [ ]:
##########
ichr = 1245  #Chromosome ID to check (Between 0 and number stated in graphs above)
##########
zxys_clean_ = zxys_clean_filtered
zxy = zxys_clean_[ichr]
fov = fovs_spots_keep[ichr]

ims_3d,zxys_im = pipeline.return_ims_pts(zxy, fov, readouts=None, master_folder=master_folders[0],
                  analysis_folder=analysis_folder, cols=['750','647','561'], pix_size=[200,109,109],window=20)

score_ = np.round(np.log(scores_clean[ichr]),2)
titles = [str(iR+1)+': '+str(score_[iR]) for iR in np.arange(len(ims_3d))]
%matplotlib inline
fig = pipeline.plot_grid_ims(ims_3d,zxys_im,titles=titles,aratio=2.5,pos_txt=6)

<h3>Separate CAST and 129 alleles</h3>

In [ ]:
import os
cell_names = []
cast_129_diff = 0.5

cell_ids = np.array(np.arange(len(zxys_clean_filtered))/2,dtype=int) # create a list of length zxys, integer divide by 2 to make doubles
cell_ids_keep = cell_ids_clean # remove all bad ones identified above (post EM step)
zyxs = zxys_clean_filtered.copy()
hs = hs_clean.copy()
hs[np.isnan(zyxs[:,:,0])]=np.nan
cids = cell_ids_keep.copy()
round_to_sort = 25

cids_,cts_= np.unique(cids,return_counts=True)
cids_kp = cids_[cts_==2] #indices of cells that have two remaining homologs
cell_ids_clean_, cids_clean_cts_ = np.unique(cell_ids_clean,return_counts=True)
cids_clean_kp = cell_ids_clean_[cids_clean_cts_==2]
hmin=np.nanmedian(hs)/2
zxys_cast = []
hs_cast = []
zxys_noncast = []
hs_noncast = []
cids_final = []
ignore = []
print("Ignoring " +str(len(ignore))+ " manually selected cells.")

for cid,cname in zip(cids_kp, cids_clean_kp):
    if cid in ignore:
        continue
    keep_homs = cids==cid
    zxy1,zxy2 = zyxs[keep_homs]
    h1s,h2s = hs[keep_homs,:]
    h1,h2 = h1s[round_to_sort],h2s[round_to_sort]
    if h1>hmin and not(h2>cast_129_diff*h1):
        zxys_cast.append(zxy1)
        hs_cast.append(h1s)
        zxys_noncast.append(zxy2)
        hs_noncast.append(h2s)
        cids_final.append(cid)
        cell_names.append(cname)
    if h2>hmin and not(h1>cast_129_diff*h2):
        cids_final.append(cid)
        zxys_cast.append(zxy2)
        hs_cast.append(h2s)
        zxys_noncast.append(zxy1)
        hs_noncast.append(h1s)
        cell_names.append(cname)
        
filename = post_analysis_folder+os.sep+"cast_allele_coords.npy"
zxys_cast_corr = np.array(zxys_cast)
np.save(filename, zxys_cast_corr)
filename = post_analysis_folder+os.sep+"129_allele_coords.npy"
np.save(filename, (np.array(zxys_noncast)))
zxys_noncast_corr = np.array(zxys_noncast)
filename = post_analysis_folder+os.sep+"cids_final.npy"
cids_final = np.save(filename,np.array(cids_final))

print("Number of cells:",len(zxys_noncast))

In [ ]:
#Save coordinates to file
p = [8,9,10]     #Segments that include the promoter (counting from 0)
e = [30,31,32]   #Segments that include the super-enhancer (counting from 0)

def get_dists(zxys):
    dists_ep = []
    for p_i in p:
        for e_i in e:
            p_ = np.nanmedian(np.array(zxys)[:,[p_i]],axis=1)#if feature spans multiple regions get the mean position
            e_ = np.nanmedian(np.array(zxys)[:,[e_i]],axis=1)
            dists_ep.append(np.linalg.norm(p_-e_,axis=-1))

    dists_ep = np.nanmedian(dists_ep,axis=0)
    return dists_ep

def get_insulation(zxys,a,b,c,func=np.nanmedian):
    distances_in_domAB_and_domBC = np.array([list(pdist(zxy_[a:b]))+list(pdist(zxy_[b:c])) for zxy_ in zxys])
    distances_between_domains = np.array([cdist(zxy_[a:b],zxy_[b:c]).ravel() for zxy_ in zxys])
    insulation = func(distances_between_domains,axis=-1)/func(distances_in_domAB_and_domBC,axis=-1)
    insulation = insulation[~np.isnan(insulation)]
    return insulation

def get_rgs(zxys,e1,e2,remove_nan=True):
    zxys_ = np.array(zxys)[:,e1:e2].copy()
    zxys_ -= np.nanmean(zxys_,axis=1)[:,np.newaxis]
    zxys_ = np.linalg.norm(zxys_,axis=-1)
    rg_cats = np.sqrt(np.nanmean(zxys_**2,axis=-1))
    if remove_nan:
        rg_cats = rg_cats[~np.isnan(rg_cats)]
    return rg_cats

ins_CAST = np.log(get_insulation(zxys_cast,9,25,33))
ins_129 = np.log(get_insulation(zxys_noncast,9,25,33))

dists_ep_cast = get_dists(zxys_cast)
dists_ep_noncast = get_dists(zxys_noncast)

rgs_cast = get_rgs(zxys_cast,10,25)
rgs_noncast = get_rgs(zxys_noncast,10,25)

cast_median = np.nanmedian(zxys_cast,axis=1)
noncast_median = np.nanmedian(zxys_noncast,axis=1)

headers = ['CAST_z', 'CAST_x', 'CAST_y', '129_z', '129_x', '129_y', 'cell_id', 'CAST_ep_dist',
           '129_ep_dist', 'CAST_ins', '129_ins', 'CAST_rgs_10_25', '129_rgs_10_25',
           'CAST_rgs_10_33', '129_rgs_10_33', 'CAST_rgs_1_42', '129_rgs_1_42', 'CAST_rgs_10_39', '129_rgs_10_39',
           'CAST_rgs_25_33', '129_rgs_25_33']
extra_data = zip(cell_names, dists_ep_cast, dists_ep_noncast, ins_CAST, ins_129,
                 get_rgs(zxys_cast,10,25), get_rgs(zxys_noncast,10,25),
                 get_rgs(zxys_cast,10,33), get_rgs(zxys_noncast,10,33),
                 get_rgs(zxys_cast,1,42), get_rgs(zxys_noncast,1,42),
                 get_rgs(zxys_cast,10,39), get_rgs(zxys_noncast,10,39),
                 get_rgs(zxys_cast,25,33), get_rgs(zxys_noncast,25,33)
                )

with open(post_analysis_folder+os.sep+'single_cell_data.csv', 'w') as f:
    print(','.join(headers), file=f)
    for cast_coords, noncast_coords, extra in zip(cast_median, noncast_median, extra_data):
        print(','.join([str(x) for x in cast_coords])+","+','.join([str(x) for x in noncast_coords])+","+','.join([str(x) for x in extra]), file=f)

<h4>Plot: Median distance heatmap for CAST alleles</h4>

In [ ]:
#########
vmin = 0  #Bottom of color scale (in nm)
vmax = 700  #Top of color scale (in nm)
#########

zxys_clean_ = copy.deepcopy(zxys_cast_corr)
mats_clean_cast = np.array(list(map(squareform,list(map(pdist,zxys_clean_)))))

im_cast = np.nanmedian(mats_clean_cast[:,:42,:42],axis=0)# take the median image for the representative heatmap

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.title('Median distance - CAST\n for '+experiment_name+'\n'+str(len(zxys_clean_))+' chromosomes', size=24)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.imshow(im_cast,vmin=vmin,vmax=vmax,cmap='seismic_r')
plt.colorbar()
fig.savefig(post_analysis_folder+'median_distance-CAST.pdf',bbox_inches='tight')
fig.savefig(post_analysis_folder+'median_distance-CAST.png',bbox_inches='tight')
np.save(analysis_folder+os.sep+r'CAST_median_matrix.npy', im_cast)

<h4>Plot: Median distance heatmap for 129 alleles</h4>

In [ ]:
#########
vmin = 0  #Bottom of color scale (in nm)
vmax = 700  #Top of color scale (in nm)
#########

zxys_clean_ = copy.deepcopy(zxys_noncast_corr)
#zxys_clean_[:,round_to_sort]=np.nan
mats_clean_129 = np.array(list(map(squareform,list(map(pdist,zxys_clean_)))))

im_129 = np.nanmedian(mats_clean_129[:,:42,:42],axis=0)# take the median image for the representative heatmap
im_129[:,round_to_sort] = np.nan
im_129[round_to_sort,:] = np.nan


fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.title('Median distance - 129\n for '+experiment_name+'\n'+str(len(zxys_clean_))+' chromosomes', size=24)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.imshow(im_129,vmin=vmin,vmax=vmax,cmap='seismic_r')
plt.colorbar()
fig.savefig(post_analysis_folder+'median_distance-129.pdf', bbox_inches='tight')
fig.savefig(post_analysis_folder+'median_distance-129.png', bbox_inches='tight')
np.save(analysis_folder+os.sep+r'129_median_matrix.npy', im_129)

<h4>Plot: Contact frequency heatmap for CAST alleles</h4>

In [ ]:
########
cutoff = 200  #Distance (nm) threshold for a contact
########

zxys_clean_ = copy.deepcopy(zxys_cast_corr)
mats_clean = np.array(list(map(squareform,list(map(pdist,zxys_clean_)))))

mat_cast = 1.*np.sum(mats_clean<cutoff,axis=0)/np.sum(mats_clean>=0,axis=0)
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.title('Contact map - CAST - cutoff:'+str(cutoff)+'nm\n for '+experiment_name+'\n'+str(len(mats_clean))+' chromosomes', size=24)
ax = plt.imshow(np.log(mat_cast),cmap='seismic')
vmin,vmax = ax.get_clim()
plt.colorbar()
fig.savefig(post_analysis_folder+'contact_map-CAST.pdf', bbox_inches='tight')
fig.savefig(post_analysis_folder+'contact_map-CAST.png', bbox_inches='tight')
fig.savefig(post_analysis_folder+'contact_map-CAST.svg', bbox_inches='tight')
np.save(analysis_folder+os.sep+r'CAST_log_contact.npy', np.log(mat_cast))

<h4>Plot: Contact frequency heatmap for 129 alleles</h4>

In [ ]:
########
cutoff = 200  #Distance (nm) threshold for a contact
########

zxys_clean_ = copy.deepcopy(zxys_noncast_corr)
zxys_clean_[:,round_to_sort]=np.nan
mats_clean = np.array(list(map(squareform,list(map(pdist,zxys_clean_)))))

mat_129 = 1.*np.sum(mats_clean<cutoff,axis=0)/np.sum(mats_clean>=0,axis=0)
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.title('Contact map - 129 - cutoff:'+str(cutoff)+'nm\nfor '+experiment_name+'\n'+str(len(mats_clean))+' chromosomes', size=24)
plt.imshow(np.log(mat_129),cmap='seismic',vmin=vmin,vmax=vmax)
plt.colorbar()
fig.savefig(post_analysis_folder+'contact_map-129.pdf', bbox_inches='tight')
fig.savefig(post_analysis_folder+'contact_map-129.png', bbox_inches='tight')
fig.savefig(post_analysis_folder+'contact_map-129.svg', bbox_inches='tight')
np.save(analysis_folder+os.sep+r'129_log_contact.npy', np.log(mat_129))

<h2>Plot: Difference of median distance (129-CAST)</h2>

In [ ]:
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.title('Difference Matrix (129-CAST)\nfor '+experiment_name+'\n'+str(len(zxys_clean_))+' chromosomes',size=24)
plt.imshow(im_129-im_cast,cmap='seismic_r',vmin=-75,vmax=75)
plt.colorbar();
fig.savefig(post_analysis_folder+'diff_of_medians.pdf', bbox_inches='tight')
fig.savefig(post_analysis_folder+'diff_of_medians.png', bbox_inches='tight')
fig.savefig(post_analysis_folder+'diff_of_medians.svg', bbox_inches='tight')
np.save(analysis_folder+os.sep+r'diff_of_medians.npy', im_129-im_cast)

<h4>Plot: Log-difference of contact frequency (CAST-129)</h4>

In [ ]:
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.ylabel('Genomic Region Number', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.title('Log-difference of contact\n (129-CAST) - '+'Cutoff:'+str(cutoff)+'nm\nfor '+experiment_name+'\n'+str(len(zxys_clean_))+' chromosomes',size=24)
plt.imshow(np.log(mat_129)-np.log(mat_cast),cmap='seismic', vmax=1.0, vmin=-1.0)
plt.colorbar()
fig.savefig(post_analysis_folder+'log_diff_contact.pdf', bbox_inches='tight')
fig.savefig(post_analysis_folder+'log_diff_contact.png', bbox_inches='tight')
fig.savefig(post_analysis_folder+'log_diff_contact.svg', bbox_inches='tight')
np.save(analysis_folder+os.sep+r'log_diff_of_contact.npy', np.log(mat_129)-np.log(mat_cast))

<h3>Single-cell analysis</h3>

<h4>Enhancer-promoter distance comparison between alleles</h4>

In [ ]:
#########
p = [9,10,11]     #Segments that include the promoter (counting from 0)
e = [30,31,32]   #Segments that include the super-enhancer (counting from 0)
#########

def get_dists(zxys):
    dists_ep = []
    for p_i in p:
        for e_i in e:
            p_ = np.nanmedian(np.array(zxys)[:,[p_i]],axis=1)#if feature spans multiple regions get the mean position
            e_ = np.nanmedian(np.array(zxys)[:,[e_i]],axis=1)
            dists_ep.append(np.linalg.norm(p_-e_,axis=-1))

    dists_ep = np.nanmedian(dists_ep,axis=0)
    dists_ep = dists_ep[~np.isnan(dists_ep)]#keep valid distances
    return dists_ep

dists_ep_cast = get_dists(zxys_cast)
dists_ep_noncast = get_dists(zxys_noncast)
savepath = post_analysis_folder+os.sep+r'cast_e_p_dists.npy'
np.save(savepath, dists_ep_cast)
savepath = post_analysis_folder+os.sep+r'noncast_e_p_dists.npy'
np.save(savepath, dists_ep_noncast)

res = ranksums(dists_ep_cast,dists_ep_noncast)
pvalue = res[-1]
pvalue

bins=np.linspace(0,800,16)
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.hist(dists_ep_cast,bins=bins,density=True,label='CAST (median = {:.0f}nm)'.format(np.nanmedian(dists_ep_cast)), edgecolor='white', linewidth=0.9);
plt.hist(dists_ep_noncast,bins=bins,density=True,alpha=0.75,label='129 (median = {:.0f}nm)'.format(np.nanmedian(dists_ep_noncast)), edgecolor='white', linewidth=0.9);
#plt.legend()
plt.title(experiment_name+" single cell \n contact for "+str(p)+' and '+str(e)+"\nWilcoxon p-value = {}".format(str(pvalue)), size=24)
plt.xlabel('Distance E-P',size=14)
plt.ylabel('Probability density',size=14)
plt.legend(['CAST = '+str(np.round(np.nanmedian(dists_ep_cast),1)), '129 = '+str(np.round(np.nanmedian(dists_ep_noncast),1)) ])
plt.savefig(post_analysis_folder+'enh_prom_distance.pdf',dpi=300,bbox_inches="tight")
plt.savefig(post_analysis_folder+'enh_prom_distance.png',dpi=300,bbox_inches="tight")
plt.savefig(post_analysis_folder+'enh_prom_distance.svg',dpi=300,bbox_inches="tight")
print(np.nanmedian(dists_ep_cast))
print(np.nanmedian(dists_ep_noncast))

<h4>Correlations of E-P dists between alleles within each cell</h4>

In [ ]:
def get_dists_nans(zxys):
    dists_ep = []
    for p_i in p:
        for e_i in e:
            p_ = np.nanmedian(np.array(zxys)[:,[p_i]],axis=1)#if feature spans multiple regions get the mean position
            e_ = np.nanmedian(np.array(zxys)[:,[e_i]],axis=1)
            dists_ep.append(np.linalg.norm(p_-e_,axis=-1))

    dists_ep = np.nanmedian(dists_ep,axis=0)
    return dists_ep

In [ ]:
from scipy.stats import pearsonr

dists_ep_cast_nans = get_dists_nans(zxys_cast)
dists_ep_noncast_nans = get_dists_nans(zxys_noncast)

#find valid pairs
valid_pairs = np.where(np.array(~np.isnan(dists_ep_cast_nans)*~np.isnan(dists_ep_noncast_nans)))
dists_ep_cast_nans = dists_ep_cast_nans[valid_pairs]
dists_ep_noncast_nans = dists_ep_noncast_nans[valid_pairs]

%matplotlib inline
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.scatter(dists_ep_noncast_nans, dists_ep_cast_nans, alpha=0.5)
plt.title(experiment_name+"\nCorrelation of CAST vs 129 E-P distance\n"+str(len(mats_clean))+' cells', size=24)
plt.ylabel("E-P Distance CAST (nm)", size=20)
plt.xlabel("E-P Distance 129 (nm)", size=20)
plt.text(2500,3000, pearsonr(dists_ep_noncast_nans, dists_ep_cast_nans)[0],size=14)
fig.savefig(post_analysis_folder+os.sep+'E-P_cast_129_corr.pdf',bbox_inches='tight')
fig.savefig(post_analysis_folder+os.sep+'E-P_cast_129_corr.png',bbox_inches='tight')
fig.savefig(post_analysis_folder+os.sep+'E-P_cast_129_corr.svg',bbox_inches='tight')

<h4>Enhancer-promoter contact comparison</h4>

In [ ]:
#########
contact_distance = 200   #Distance (nm) threshold for contact
#########

cast_fraction = 1.*np.sum(dists_ep_cast<contact_distance)/np.sum(dists_ep_cast>=0)
noncast_fraction = 1.*np.sum(dists_ep_noncast<contact_distance)/np.sum(dists_ep_noncast>=0)
#Fisher's exact test for contact < contact_distance (nm)
from scipy.stats import fisher_exact
cast_contact = np.sum(dists_ep_cast<=contact_distance)
cast_none = np.sum(dists_ep_cast>contact_distance)
wt_contact = np.sum(dists_ep_noncast<=contact_distance)
wt_none = np.sum(dists_ep_noncast>contact_distance)
print(f"\t<={contact_distance}\t>{contact_distance}")
print(f"CAST\t{cast_contact}\t{cast_none}")
print(f"129\t{wt_contact}\t{wt_none}")
table = [[cast_contact,cast_none],[wt_contact,wt_none]]
oddsratio, pvalue = fisher_exact(table)
print("\nFisher exact test")
print(f"Odds ratio: {oddsratio}")
print(f"P-value: {pvalue}")

In [ ]:
#########
minimum = 200
maximum = 750
step = 25
#########

import seaborn as sns

def enh_prom_distance_ratio(threshold):
    cast_contact = np.sum(dists_ep_cast<=threshold)
    cast_none = np.sum(dists_ep_cast>threshold)
    wt_contact = np.sum(dists_ep_noncast<=threshold)
    wt_none = np.sum(dists_ep_noncast>threshold)
    return (float(cast_contact) / (cast_contact+cast_none)) / (float(wt_contact) / (wt_contact+wt_none))

dists = list(range(minimum,maximum,step))
ratio_nums = [enh_prom_distance_ratio(thresh) for thresh in dists]
#smoothing
#smooth = [sum(ratio_nums[i-25:i+26]) / 51.0 for i in range(105,650)]
fig,ax = plt.subplots(figsize=(6,5))
plt.xlabel("E-P contact threshold (nm)",size=14)
plt.ylabel("CAST/129 contact frequency ratio",size=14)
plt.title(experiment_name+"\nE-P constact ratio (CAST/129)", size=24)
plot = sns.lineplot(dists, ratio_nums)
plt.show(plot)
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=8)

savefile = post_analysis_folder+os.sep+"E-P_ratio_line.npy"
np.save(savefile, np.array(list(zip(ratio_nums, dists))))

plt.savefig(post_analysis_folder+'ep_threshold.pdf',dpi=300,bbox_inches="tight")
plt.savefig(post_analysis_folder+'ep_threshold.png',dpi=300,bbox_inches="tight")
plt.savefig(post_analysis_folder+'ep_threshold.svg',dpi=300,bbox_inches="tight")

<h4>Radius of gyration</h4>

In [ ]:
########
regions_to_plot = [(9,33), (9,25), (26,33), (26,42), (0,25), (0,42)]
########

def get_rgs(zxys_cast_,remove_nan=True):
    zxys_cast_ -= np.nanmean(zxys_cast_,axis=1)[:,np.newaxis]
    zxys_cast_ = np.linalg.norm(zxys_cast_,axis=-1)
    rg_cats = np.sqrt(np.nanmean(zxys_cast_**2,axis=-1))
    if remove_nan:
        rg_cats = rg_cats[~np.isnan(rg_cats)]
    return rg_cats


def histogram_rgs(zxys_cast,zxys_noncast,edges=[9,33],bins=np.linspace(50,500,16)):
    e1,e2 = edges
    zxys_cast_ = np.array(zxys_cast)[:,e1:e2].copy()
    rgs_cast = get_rgs(zxys_cast_)
    zxys_noncast_ = np.array(zxys_noncast)[:,e1:e2].copy()
    rgs_noncast = get_rgs(zxys_noncast_)
    fig = plt.figure(figsize=(5.5,4))
    ax = fig.add_subplot(111)
    plt.setp(ax.spines.values(), linewidth=1.5)
    ax.tick_params(width=1.5, length=4, labelsize=14)
    
    plt.hist(rgs_cast,bins=bins,density=True,alpha=0.75,label='CAST - median(nm):'+str(int(np.median(rgs_cast))), edgecolor='white', linewidth=0.9)
    plt.hist(rgs_noncast,bins=bins,density=True,alpha=0.75,label='129 - median(nm):'+str(int(np.median(rgs_noncast))), edgecolor='white', linewidth=0.9)
    #plt.legend()
    plt.ylabel("Probability density",size=14)
    plt.xlabel("Radius of gyration(nm)",size=14)
    res = ranksums(rgs_cast,rgs_noncast)
    pvalue = res[-1]
    plt.title('Radius of gyration of regions '+str(e1+1)+'-'+str(e2)+'\n'+experiment_name+'\np-value Wilcoxon: '+str(pvalue), size=24)
    fig.savefig(post_analysis_folder+'radiusgyration_'+str(e1+1)+'-'+str(e2)+'.pdf',dpi=300,bbox_inches="tight")
    fig.savefig(post_analysis_folder+'radiusgyration_'+str(e1+1)+'-'+str(e2)+'.png',dpi=300,bbox_inches="tight")
    fig.savefig(post_analysis_folder+'radiusgyration_'+str(e1+1)+'-'+str(e2)+'.svg',dpi=300,bbox_inches="tight")
    return fig,rgs_cast,rgs_noncast

for region in regions_to_plot:
    histogram_rgs(zxys_cast,zxys_noncast,edges=region,bins=np.linspace(50,500,16));

<h4>Insulation of domains</h4>

In [ ]:
#########
#Insulation of regions start-middle and middle-end
start = 9
middle = 25
end = 33
#########

def get_insulation(zxys,a,b,c,func=np.nanmedian):
    distances_in_domAB_and_domBC = np.array([list(pdist(zxy_[a:b]))+list(pdist(zxy_[b:c])) for zxy_ in zxys])
    distances_between_domains = np.array([cdist(zxy_[a:b],zxy_[b:c]).ravel() for zxy_ in zxys])
    insulation = func(distances_between_domains,axis=-1)/func(distances_in_domAB_and_domBC,axis=-1)
    insulation = insulation[~np.isnan(insulation)]
    return insulation

def insulation_figure(zxys_cast,zxys_noncast,a,b,c,bins = np.linspace(-1,1,30)):
    ins_CAST = np.log(get_insulation(zxys_cast,a,b,c))
    ins_129 = np.log(get_insulation(zxys_noncast,a,b,c))
    fig =  plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
    plt.setp(ax.spines.values(), linewidth=1.5)
    ax.tick_params(width=1.5, length=4, labelsize=14)
    plt.hist(ins_CAST,density=True,bins=bins,alpha=0.75,label='CAST - median:'+str(np.round(np.median(ins_CAST),2)), edgecolor='white', linewidth=0.9)
    plt.hist(ins_129,density=True,bins=bins,alpha=0.75,label='129 - median:'+str(np.round(np.median(ins_129),2)), edgecolor='white', linewidth=0.9)
    plt.xlabel('Log-insulation score\n (median distances between/median distance within domains)',size=14)
    plt.ylabel('Prbability density\nacross cells',size=14)
    #plt.legend()
    res = ranksums(ins_CAST,ins_129)
    pvalue = res[-1]
    
    plt.title('Single cell insulation of domains '+str((a+1,b))+'- '+str((b+1,c))+'\np-value -wilcoxon:'+str(pvalue),size=24)
    fig.savefig(post_analysis_folder+'insulation.pdf',dpi=300,bbox_inches="tight")
    fig.savefig(post_analysis_folder+'insulation.png',dpi=300,bbox_inches="tight")
    fig.savefig(post_analysis_folder+'insulation.svg',dpi=300,bbox_inches="tight")
    return fig,ins_CAST,ins_129

zxys_cast = np.array(zxys_cast)
zxys_noncast = np.array(zxys_noncast)

fig,ins_CAST,ins_129 = insulation_figure(zxys_cast,zxys_noncast,start,middle,end)

In [ ]:
import DomainTools as dt
dom_sz = 6
cutoff = 0.33
dom_starts_all_cast = []
for zxy_ in (zxys_cast):

    bad = np.where(np.isnan(zxy_[:,0]))[0]
    zxy__ = np.array([dt.interp1dnan(x_) for x_ in zxy_.T]).T

    zxy___ = zxy__
    
    #get initial guess of where a domains boundary is
    dists = np.array([np.linalg.norm(np.nanmean(zxy___[max(i-dom_sz,0):i],axis=0)-\
                            np.nanmean(zxy___[i:i+dom_sz],axis=0)) 
                              for i in range(len(zxy___))])
    bds = dt.get_ind_loc_max(dists,cutoff_max=0,valley=dom_sz)
    dom_starts= [0]+[dm for dm in bds if dm>1 and dm<len(zxy___)-2]
    mat = squareform(pdist(zxy___))
    dom_starts_ = dom_starts
    # fuse domains untill cannot fuse anymore based on cutoff.
    if len(dom_starts)>1:
        dom_starts_,seps = dt.fuse_doms(mat,dom_starts,tag='median',cut_off=cutoff)
    dom_starts_all_cast+=[dom_starts_[1:]]
    if False:
        plt.figure()
        plt.imshow(mat,vmax=800,cmap='seismic_r')
        plt.plot(dom_starts_,dom_starts_,'go')
        
dom_starts_all_noncast = []
for zxy_ in (zxys_noncast):

    bad = np.where(np.isnan(zxy_[:,0]))[0]
    zxy__ = np.array([dt.interp1dnan(x_) for x_ in zxy_.T]).T

    zxy___ = zxy__
    dists = np.array([np.linalg.norm(np.nanmean(zxy___[max(i-dom_sz,0):i],axis=0)-\
                            np.nanmean(zxy___[i:i+dom_sz],axis=0)) 
                              for i in range(len(zxy___))])
    bds = dt.get_ind_loc_max(dists,cutoff_max=0,valley=dom_sz)
    dom_starts= [0]+[dm for dm in bds if dm>1 and dm<len(zxy___)-2]
    mat = squareform(pdist(zxy___))
    dom_starts_ = dom_starts
    if len(dom_starts)>1:
        dom_starts_,seps = dt.fuse_doms(mat,dom_starts,tag='median',cut_off=cutoff)
    dom_starts_all_noncast+=[dom_starts_[1:]]
    if False:
        plt.figure()
        plt.imshow(mat,vmax=800,cmap='seismic_r')
        plt.plot(dom_starts_,dom_starts_,'go')

In [ ]:
bds_all = [e for dm in dom_starts_all_cast for e in dm]
fig = dt.fig_no_axis(figsize=(10,6))
fracs = np.bincount(bds_all)/float(len(dom_starts_all_cast))
fracs[:dom_sz-1]=np.nan
fracs[-dom_sz+1:]=np.nan
#fracs = np.insert(np.delete(fracs, 25), 38, np.nan)
np.save(post_analysis_folder+ os.sep+r"boundary_prob_cast.npy", fracs)
plt.plot(fracs,'.-',label='CAST', linewidth=8)

bds_all = [e for dm in dom_starts_all_noncast for e in dm]
fracs = np.bincount(bds_all)/float(len(dom_starts_all_noncast))
fracs[:dom_sz-1]=np.nan
fracs[-dom_sz+1:]=np.nan
#fracs = np.insert(np.delete(fracs, 25), 38, np.nan)
np.save(post_analysis_folder+ os.sep+r"boundary_prob_129.npy", fracs)
plt.plot(fracs,'.-',label='129', linewidth=8)

plt.xticks(np.arange(fracs.shape[0],step=5), np.arange(1,fracs.shape[0]+1,step=5))
plt.ylabel('Boundary probability',size=14)
plt.xlabel('Genomic coordinate',size=14)
#plt.axvline(25,linestyle='--', color='black',linewidth=2)
plt.ylim(0,0.20)
plt.title("Boundary Probability for\n"+experiment_name)
plt.legend()
filename = post_analysis_folder+os.sep+r'\Bondary_prob_'+experiment_name[:5]+'.pdf'
plt.savefig(filename, format='pdf', dpi=400, bbox_inches='tight')
filename = post_analysis_folder+os.sep+r'\Bondary_prob_'+experiment_name[:5]+'.png'
plt.savefig(filename, format='png', dpi=400, bbox_inches='tight')

In [ ]:
import numpy as np 
        
def centeroidnp(trace):
    length = trace.shape[0]
    sum_x = np.nansum(trace[:, 1])
    sum_y = np.nansum(trace[:, 2])
    sum_z = np.nansum(trace[:, 0])
    return sum_z/(length-sum(np.isnan(trace[:, 0]))), sum_x/(length-sum(np.isnan(trace[:, 1]))), sum_y/(length-sum(np.isnan(trace[:, 2])))

def get_dists_from_ctr(array):

    geometric_distances = np.zeros((array.shape[0], array.shape[1]))

    # go through each trace
    for idx, trace in enumerate(array):

            #get trace centeroid
        trace_center = np.array(centeroidnp(trace))

            # compute distance between centeroid and each point (returns a 42 element array for each trace)
        centeroid_distances = np.array([ np.linalg.norm(x) for x in (trace-trace_center) ])

        geometric_distances[idx] = centeroid_distances

    return geometric_distances
    
cast = get_dists_from_ctr(zxys_cast_corr)
noncast = get_dists_from_ctr(zxys_noncast_corr)
noncast[:,25] = np.nan

fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(111)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.tick_params(width=1.5, length=4, labelsize=14)
plt.title('Median distance from geometric center for\n'+experiment_name+", "+str(len(cast))+' chromosomes', size=24)
plt.ylabel('Median Distance from geometric center', size=14)
plt.xlabel('Genomic Region Number', size=14)
plt.ylim((100,max(np.nanmedian(noncast, axis=0))+50))
plt.plot(range(42), np.nanmedian(cast, axis=0), marker='o', color='#1f77b4')
plt.errorbar(range(42), np.nanmedian(cast, axis=0), yerr=1.96*np.nanstd(cast)/np.sqrt(sum(np.isnan(cast))), color='#1f77b4')
plt.plot(range(42), np.nanmedian(noncast, axis=0), marker='o', color='#ff7f0e')
plt.errorbar(range(42), np.nanmedian(noncast, axis=0), yerr=1.96*np.nanstd(noncast)/np.sqrt(sum(np.isnan(noncast))), color='#ff7f0e')
plt.vlines([9,25, 31], ymin=100, ymax=max(np.nanmedian(noncast, axis=0))+45, colors='r', alpha=0.3, linewidth=10)
plt.legend(['CAST', '129'])
fig.savefig(post_analysis_folder+os.sep+'distance_geom_center.pdf',bbox_inches='tight')
fig.savefig(post_analysis_folder+os.sep+'distance_geom_center.png',bbox_inches='tight')
fig.savefig(post_analysis_folder+os.sep+'distance_geom_center.svg',bbox_inches='tight')
# save plot data.
np.save(analysis_folder+os.sep+r'cast_geom_distance_median.npy', np.nanmedian(cast, axis=0))
np.save(analysis_folder+os.sep+r'129_geom_distance_median.npy', np.nanmedian(noncast, axis=0))
np.save(analysis_folder+os.sep+r'cast_geom_distance_errors.npy', 1.96*np.nanstd(cast)/np.sqrt(sum(np.isnan(cast))))
np.save(analysis_folder+os.sep+r'129_geom_distance_errors.npy', 1.96*np.nanstd(noncast)/np.sqrt(sum(np.isnan(noncast))))